In [ ]:
import httpx
import requests
import pandas as pd
from tqdm import tqdm

In [ ]:
# async with httpx.AsyncClient() as client:
#     overall_data = await client.get('https://fantasy.premierleague.com/api/bootstrap-static/')
overall_data = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/')
overall_data = overall_data.json()

In [ ]:
df = pd.DataFrame(overall_data['elements'])

In [ ]:
df.index=df['id']

In [ ]:
columns_to_remove = ['first_name','photo','news','news_added','second_name','squad_number','web_name','code']

In [ ]:
df.drop(columns_to_remove,axis=1,inplace=True)
len(df.columns)

In [ ]:
df = df[(df.status != "u")]
len(df)

In [ ]:
train = []
test = []
for i in tqdm(range(len(df['id']))):
    player_data = requests.get('https://fantasy.premierleague.com/api/element-summary/'+str(df['id'].iloc[i])+'/')
    player_data = player_data.json()
    for i in player_data['history']:
        train.append(i)
    for i in player_data['fixtures']:
        test.append(i)